### Lab 02
Name: Marie Nguyen 
#### Part 1:

Modify your code for lab 1 so that it:

a. parses only the headline and text sections of each document.

b. can run with stopping and/or stemming either on or off

In part 1, I am going to parse document number, headline and text from the XML file collection and write these elements into a file for comparision when stopping or stemming is gradually on or off. 

In [1]:
# Loading packages
import string
import xml.etree.ElementTree as ET
# Sources: https://docs.python.org/3/library/xml.etree.elementtree.html 

import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# Resource: https://realpython.com/nltk-nlp-python/#stemming

# Define a function for viewing all deadlines and texts of the collection when stopping is on
def stopping_file(input, output):
    """ Parameters: input: original headlines and texts from the collection
    output: document with all headlines and text removed stopwords"""
    
    stopwords_array = []
    
    # Open and read the stopword file
    with open("stoplist", "r") as stoplistFile:
        stopwords_array = stoplistFile.read().lower().split() #Tokenization
    
    with open(input, "r") as input_file, open(output, "w") as output_file:
        lines = input_file.readlines() 
        for line in lines:
            # Remove punctuation with a space (when its length = puctuation's length)
            punctTable = str.maketrans(string.punctuation, " " * len(string.punctuation))
            line = line.translate(punctTable)
            # Tokenization to create a list of words 
            words = line.split() 
            # Interate each words in the list of words
            for word in words:
            # Check if the word is not in the stop words array
                if word not in stopwords_array:
                    # Add the word to the output file if it's not a stopword
                    output_file.write(word + " ")
            output_file.write("\n")
        
    # Close the files
    input_file.close()
    output_file.close()
    
# Define a function for viewing all deadlines and texts of the collection when stemming is on
def stemming_file(input, output):
    with open(input, "r") as input_file, open(output, "w") as output_file:
        lines = input_file.readlines()
        for line in lines:
            # Remove punctuation with a space (when its length = puctuation's length)
            punctTable = str.maketrans(string.punctuation, " " * len(string.punctuation))
            line = line.translate(punctTable)
            # Tokenization to create a list of words 
            words = line.split() 
            # Interate each words in the list of words
            for word in words:
                word = stemmer.stem(word) #Nomalization
                output_file.write(word + " ")
            output_file.write("\n")

    # Close the files
    input_file.close()
    output_file.close()

# Define a parsing function for parsing headline and text of each doc 
def parsing(input_XML, healine_text_document, stopping, stemming):
    """ Parameter: input XML file, 
    headline_text_document: a document contains doc number, headline and texts 
        of the collection for comparision. 
    stopping; True or False, whether stopping is on or off
    stemming: True or False, whether stemming is on or off
    Return: A document with all document numbers, headlines, and texts 
    Use ElementTree module to read the XML file """

    tree = ET.parse(input_XML) 
    root = tree.getroot()
    
    """ Parsing only the headline and text sections of each document
        and save them in a new document """

    with open(healine_text_document, "w") as output_file:
        # Loop through each <DOC> element
        for doc in root.findall("DOC"):
            # Find the headline and text elements
            headline = doc.find("HEADLINE")
            text = doc.find("TEXT")
            # Find document number element
            doc_number = doc.find("DOCNO") 

            # Write document number, headline and text into the output file
            if headline is not None and text is not None:
                output_file.write(doc_number.text)
                output_file.write("\n")
                output_file.write(headline.text)
                output_file.write(text.text)
                output_file.write("\n")
        
    output_file.close()
    
    if stopping == True:
        stopping_file(healine_text_document, "Parsing_Stopping.txt")
        
    if stemming == True:
        stemming_file(healine_text_document, "Parsing_Stemming.txt")

# When stopping is on        
parsing("trec.sample.xml", "Original_parsing.txt", True, False)
# When stemming is on
parsing("trec.sample.xml", "Original_parsing.txt", False, True)
# When stopping and stemming is off
parsing("trec.sample.xml", "Original_parsing.txt", False, False)


#### Part 2:
Implement an inverted index for the documents. You need to save the following information for each term:

a. term (pre-processed) and it’s document frequency

b. list of documents in which this term occurred. (a linked list to store this data)

In [308]:
# Create a linked list 
# Source: https://www.geeksforgeeks.org/python-linked-list/ and Data Structure course

# Initialize a Node class to create a node 
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# Initialize a Linkedlist class
class LinkedList:
    def __init__(self):
        self.head = None

    def add(self, data):
        # Initialize a new node with the data 
        new_node = Node(data)
        # If the Linkedlist is empty, let the new node is the head 
        if not self.head:
            self.head = new_node
            return
        # Otherwise, let the current node is the head 
        current = self.head
        # Loop through the end of the Linkedlist 
        while current.next:
            current = current.next
        # Add a new node to the end. 
        current.next = new_node
                   
    def toString(self):
        # Assign a current node as the head 
        current = self.head
        # Initialize an empty string for printing out the linkedlist
        content = " "
        # Loop through all nodes in the linked list
        while (current != None):
            # Add the data of the node to the content 
            content += "    " + str(current.data) + " \n "
            # Move to the next node 
            current = current.next
            
        # content += "None"
        return content 

In [238]:
with open("stoplist", "r") as stoplistFile:
    stopwords_array = stoplistFile.read().lower().split() #Tokenization
        
# Define a function to return a dict of terms and doc frequency 
def parsing_terms(input_XML, stopping, stemming):
    # Parameter: input XML file, either stopping or stemming is on or off
    # Return: Lists of headlines and texts 
    # Use ElementTree module to read the XML file 
    
    stemmer = PorterStemmer()

    tree = ET.parse(input_XML) 
    root = tree.getroot()
    
    # Initialize a dictionary for term and its document frequency 
    term_doc_dict = {}
    
    """ Parsing only the headline and text sections of each document
        and save them in a new document """
        
    # Loop through each <DOC> element
    for doc in root.findall("DOC"):
        terms_array = []
        
        # Find the headline and text elements
        headline = doc.find("HEADLINE")
        text_doc = doc.find("TEXT")
        # Find document number element 
        doc_number = doc.find("DOCNO") 
        
        # Remove punctuation and apply lower case
        punctTable = str.maketrans(string.punctuation, " " * len(string.punctuation))
        headline = headline.text.lower().translate(punctTable)
        text_doc = text_doc.text.lower().translate(punctTable)
            
        # Get unique terms from headline in each document 
        for term in headline.split():
            if term not in terms_array:
                # On stemming 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming      
                elif stemming == True:
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original 
                else:
                    terms_array.append(term)
        
        # Get unique terms from text in each document 
        for term in text_doc.split():
            if term not in terms_array:
                # On stopping 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming        
                elif stemming == True:
                    term = stemmer.stem(term) #Nomalization
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original     
                else:
                    terms_array.append(term)
        
        # Loop through the terms array in each document 
        for word in terms_array:
            """ If the word exists in the document
            add the document number to the word's linkedlist """
            if word in term_doc_dict:
                term_doc_dict[word].add(doc_number.text)
            else:
                term_doc_dict[word] = LinkedList()
                term_doc_dict[word].add(doc_number.text)
                       
    return term_doc_dict     

#### Part 3: 

Print out for visualization, the output of your inverted index in a text file.
Example output can be found in trec.index.txt. Note that this output file was generated without applying stopping or stemming. Also notice that the output is sorted by term. Check your output inverted index when you enable/disable the following:

a. stopping

b. stemming

In [239]:
from collections import OrderedDict
# Resource: https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/ 

# Write in a document the output of the inverted index in a text file 
with open("inverted_index_file", "w") as file:
    # Call the parsing_terms function in part 2:
    term_dictionary = parsing_terms("trec.sample.xml", False, False)
    terms_list = OrderedDict(sorted(term_dictionary.items()))

    for term in terms_list:
        file.writelines(term + ":" + "\n")
        file.writelines("" + term_dictionary[term].toString())  
        
with open("inverted_index_file_stopping", "w") as file:
    # Call the parsing_terms function in part 2:
    term_dictionary = parsing_terms("trec.sample.xml", True, False)
    terms_list = OrderedDict(sorted(term_dictionary.items()))

    for term in terms_list:
        file.writelines(term + ":" + "\n")
        file.writelines("" + term_dictionary[term].toString())  
        
with open("inverted_index_file_stemming", "w") as file:
    # Call the parsing_terms function in part 2:
    term_dictionary = parsing_terms("trec.sample.xml", False, True)
    terms_list = OrderedDict(sorted(term_dictionary.items()))

    for term in terms_list:
        file.writelines(term + ":" + "\n")
        file.writelines("" + term_dictionary[term].toString())  

#### Part 4:

a) Implement a simple word overlap retrieval algorithm using the similarity function. Run the queries in the file queries.lab2.txt

In [240]:
# This code is for preprocessing step: tokenization of the queries.lab2.txt file 
with open ("queries.lab2.txt", "r") as query_file:
    queries = query_file.readlines()
    punctTable = str.maketrans("","",string.punctuation)
    for query in queries:
        query = query.strip().translate(punctTable).lower().split()
        print(query)    
query_file.close()    

['q1', 'scotland']
['q2', 'window']
['q3', 'replacing']
['q4', 'window', 'replacing']
['q5', 'condemning']
['q6', 'income', 'taxes']
['q7', 'middle', 'east', 'peace']


In [242]:
# Define a function for a dictionary of terms from the query and their LinkedList docIDs
def similarity_search():
    query_dictionary = {}
    # Call the parsing_terms function in part 2:
    term_dictionary = parsing_terms("trec.sample.xml", False, False)
    
    # Tokenize the query document into words 
    with open ("queries.lab2.txt", "r") as query_file,open("output", "w") as output_file:
        queries = query_file.readlines()
        punctTable = str.maketrans("","",string.punctuation)  # Remove punctuation 
        # Loop through each query row in the file 
        for query in queries:
            query = query.strip().translate(punctTable).lower().split() # Preprocessing 
            # Loop through terms in the query. [Ignore the first element in the query array cause it's not the term]
            for index in range(1,len(query)):
                if query[index] in term_dictionary:
                    term = query[index]
                    # Assign the LinkedList ID of the term in the inverted file to the query dictionary 
                    query_dictionary[term] = term_dictionary[term] 
                # If there is a new term, assign an empty LinkedList 
                else:
                    query_dictionary[term] = LinkedList()
    
    return query_dictionary

# Write the do
with open("test.txt", "w") as file:
    # Call the parsing_terms function in part 2:
    query_dict = similarity_search()
    # Print out the dictionary. 
    for term in query_dict:
        file.writelines(term + ":" + "\n")
        file.writelines("" + query_dict[term].toString())  

c) Implement Boolean search using AND, OR, and NOT. Run the queries in the file queries.boolean.lab2.txt.

I am stuck on this part. I will go to office hours to ask for help and finish it later on. 

In [ ]:
# Define a functionn for getting the list of document ID from the collection 
def all_docIDs(input_XML):
    tree = ET.parse(input_XML) 
    root = tree.getroot()
    
    docIDs_array = LinkedList()
    
    # Loop through each <DOC> element
    for doc in root.findall("DOC"):
        # Find document number element 
        doc_number = doc.find("DOCNO") 
        docIDs_array.add(doc_number.text)
        
    return docIDs_array

Resources: https://www.geeksforgeeks.org/evaluation-of-expression-tree/

https://www.geeksforgeeks.org/convert-infix-expression-to-postfix-expression/

I asked Astrid Zhao for help in this problem. 


In [321]:
class TreeNode:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

# Define an expression tree for the given query 
def expression_tree(query):
    # Parameter: an array of tokens in the infix query
    """ Convert the infix query to the postfix query and then build an expression tree
    so that parent nodes are logical expressions and child nodes are terms in the dictionary"""
    
    # Put a score to each logical expression to classify top nodes of building the tree 
    logical_score = {"not": 3, "or": 2, "and": 1}
    # Initialize stacks for postfix expression and logical expression stack 
    postfix = []
    logic_stack = [] 
    
    # Loop through each token the infix expression stack 
    for token in query:
        # If it is a term, add it to the posfix stack 
        if token not in logical_score: 
            postfix.append(token)
        # If it is a logical expression, add it to the logic stack 
        elif token == "not":
            logic_stack.append(token)
        else:  # (AND or OR)
            # while the logical expression stack is not empty and
            # the top element of the stack is not NOT
            while (logic_stack and logic_stack[-1] != "not"):
                # if the top element of the stack has a higher or equal score than 
                # the current token (preferred "AND" in this case)
                if logical_score[logic_stack[-1]] >= logical_score[token]:
                    # Pop the top logical expression from logic_stack and appends it to the postfix expression 
                    # To ensure token with higher score are evaluated first 
                    postfix.append(logic_stack.pop())
                    
            # Add the current token to the logic stack 
            logic_stack.append(token)
            
    # While there is still logical expression in the logic stack 
    # Remove (.pop) it from the stack and add it to the postfix stack 
    while logic_stack:
        postfix.append(logic_stack.pop())
        
    # Initialize an empty stack for building a tree 
    stack = []
    logical_expressions = ["not", "or", "and"]

    # If token is a term push that into the stack
    # If token is an logical expression pop two values from the stack and 
    # make the 2 values become children node of the logical expression node 
    # and push the current node into the stack.
    for token in postfix:
        # If the term is different from logical expression
        if token not in logical_expressions:
            # Create a term tree node and add to the stack
            stack.append(TreeNode(token))
        else:
            if token == "not":
                # get the top element from the stack
                left_term = stack.pop()
                # Create a "not" tree node 
                new_treenode = TreeNode(token)
                # Assign the term as the right node of "not" node 
                new_treenode.left = left_term
                
            else:
                # If token is "and" or "or"
                # Get the 2 top element from the stack 
                right_term = stack.pop()
                left_term = stack.pop()
                # Create a tree node for "and" or "or"
                new_treenode = TreeNode(token)
                # Assign the 2 consecutive terms as the right and left node 
                new_treenode.right = right_term 
                new_treenode.left = left_term 
                
                
            # Add the logical expression tree node to the stack 
            stack.append(new_treenode)
                
    # Return the root node of the expression tree 
    return stack[0]

# Define an or boolean search
def Boolean_search_or(term1_docIDs, term2_docIDs):
    # Parameters: Linkedlist docIDs of the 2 childs term 
    # Initialize a result LinkedList
    merge_posting = LinkedList()  
    
    current_docID_term1 = term1_docIDs.head    # Current pointer 
    current_docID_term2 = term2_docIDs.head    # Current pointer 
    
    # Loop through the 2 LinkedList at the same time 
    while current_docID_term1 != None and current_docID_term2 != None:
        # If docID of the first LinkedList < docID of the second LinkedList 
        if current_docID_term1.data < current_docID_term2.data:
            # Add that docID of the first LinkedList to the result posting 
            merge_posting.add(current_docID_term1.data)
            # Moving the pointer in the first LinkedList 
            current_docID_term1 = current_docID_term1.next
        # If docID of the second LinkedList < docID of the first LinkedList    
        elif current_docID_term2.data < current_docID_term1.data:
            # Add that docID of the second LinkedList to the result posting 
            merge_posting.add(current_docID_term2.data)
            # Moving the pointer in the second LinkedList 
            current_docID_term2 = current_docID_term2.next
            
        else:
            # Both LinkedLists have the same docID, add that docID to the result posting 
            merge_posting.add(current_docID_term2.data)
            # Moving the pointers in the 2 LinkedLists
            current_docID_term1 = current_docID_term1.next
            current_docID_term2 = current_docID_term2.next
            
    return merge_posting

# Define an and boolean search 
def Boolean_search_and(term1_docIDs, term2_docIDs):
    # Parameters: Linkedlist docIDs of the 2 childs term 
    # Initialize a result LinkedList
    merge_posting = LinkedList()
    
    current_docID_term1 = term1_docIDs.head   # Current pointer 
    current_docID_term2 = term2_docIDs.head   # Current pointer 
    
    # Loop through the first LinkedList: 
    while current_docID_term1 != None: 
        current_docID_term2 = term2_docIDs.head   # Current pointer
        # Loop through the second LinkedList 
        while current_docID_term2 != None:
            # Both LinkedLists have the same docID, add that docID to the result posting 
            if current_docID_term1.data == current_docID_term2.data:
                merge_posting.add(current_docID_term1.data)
                break
            # Moving the pointer in the second LinkedList
            current_docID_term2 = current_docID_term2.next
        # Moving the pointer in the first LinkedList    
        current_docID_term1 = current_docID_term1.next
            
    return merge_posting

def Boolean_search_not(term_docIDs):
    # Parameters: Linkedlist docIDs of the "not"'s child term 
    # Call the LinkedList of all docIDs collection 
    all_docID_list = all_docIDs("trec.sample.xml")
    # Initialize an empty LinkedList to store docIDs that not in the term's docIDs 
    posting = LinkedList()
    
    current_docID_term = term_docIDs.head   # Current pointer 
    current_all_docIDs = all_docID_list.head   # Current pointer 

    # Loop through the all docIDs collection 
    while current_all_docIDs != None:
        # Compare docID in the term LinkedList to docID in the collection 
        if current_docID_term == None or current_docID_term.data == None or current_all_docIDs.data < current_docID_term.data:
            # Add that docID to the result LinkedList 
            posting.add(current_all_docIDs.data)
            # Moving the pointer of the all docIDs collection 
            current_all_docIDs = current_all_docIDs.next
            
        # Both LinkedLists have the same docID, skip this docID 
        elif current_all_docIDs.data == current_docID_term.data:
            # Moving the pointers in the 2 LinkedLists 
            current_docID_term = current_docID_term.next
            current_all_docIDs = current_all_docIDs.next
            
        else:
            # Moving the pointer in the term LinkedList 
            current_docID_term = current_docID_term.next

    return posting

# Define a function to evaluate the expression tree
def Boolean_retrieval_expression_tree(root):
    # Parameter: the root of the expression tree 
    query_dictionary = similarity_search() 
    
    # Return non if it is an empty tree 
    if root is None:
        return LinkedList()

    # Apply the operator represented by the current node
    if root.data == "and":
        # Evaluate left and right subtrees
        left_result = Boolean_retrieval_expression_tree(root.left)
        right_result = Boolean_retrieval_expression_tree(root.right)
        # Evaluate the "AND"
        return Boolean_search_and(left_result, right_result)
    
    elif root.data == "or":
        # Evaluate left and right subtrees
        left_result = Boolean_retrieval_expression_tree(root.left)
        right_result = Boolean_retrieval_expression_tree(root.right)
        # Evaluate the "OR"
        return Boolean_search_or(left_result,right_result)
    
    elif root.data == "not":
        # Evaluate the right subtree:
        right_result = Boolean_retrieval_expression_tree(root.right)
        # Evaluate the "NOT "
        return Boolean_search_not(right_result) 
        
    else: # Base case
        # Leaf node, return the corresponding linked list
        term = root.data
        return(query_dictionary[term])


with open ("queries.boolean.lab2.txt", "r") as query_file, open("testBoolean.txt", "w") as output_file:
    queries = query_file.readlines()
    punctTable = str.maketrans("","",string.punctuation) # Remove punctuation 
    # Read each querry line in the queries.lab2.txt doc
    for query in queries:
        # Tokenize the query into words with lower case and without punctuation 
        query = query.strip().lower().translate(punctTable).split()
        
        output_file.writelines(query[0])
        output_file.writelines("\n")
        # Initialize a new query array containing terms and logical expression 
        new_query = []
        for index in range(1,len(query)):
            new_query.append(query[index])
            
        # Initialize an expression tree given the new query 
        root = expression_tree(new_query)
        
        # Calling the Boolean retrieval function for a list of docIDs 
        posting_IDs = Boolean_retrieval_expression_tree(root)
        
        # Write the result list into the output file.
        output_file.writelines(posting_IDs.toString()) 
         

d) Report the list of documents retrieved for each query.

i. How do the word overlap results compare to the Boolean results?

When there is only 1 term in the query, the result of the word overlap and the Boolean is the same. Otherwise, the word overlap results all document IDs that contain the terms in the query while Boolean search return only document IDs that exactly match the query. Taken "q4: window AND replacin"g as an example, the word overlap results in document IDs having the 2 terms (window and replacing) while the Boolean search results only 1 document. Hence, the Boolean retrieval system is stricter than the word overlap retrieval system. 

ii. Where it makes sense, compare and contrast the results for different versions of the same query between the Boolean retrieval and word overlap retrieval (e.g. “middle AND east AND peace” versus “middle east peace”)

The result from the Boolean query "middle AND east AND peace" will sort out and retrieve document IDs that contain all 3 terms while the word overlap query "middle east peace" will consider the similarity score of documents to the query retrieve documents that closely match the query's terms. Hence, the Boolean retrieval results will be more precise while the results of the word overlap retrieval is flexibile including partial match documents. 

e) These collections are quite small. What do you expect would happen if you run your system as implemented on a very large collection?

If we have a very large collection, both computing similarity function in word overlap retrieval and returning the posting list of document ID in Boolean retrieval systems will take longer time and can be computationally expensive. In addition, my computer might not have enough storage to store the Linkedlist document IDs of terms and preprocessing such as stemming and stopping might take more time to proceed. 